# GoogLeNet V4

<img src="imgs/googlenetv4.png">

<img src="imgs/googlenetv4*.png">

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [39]:
class Stem(nn.Module):
    def __init__(self):
        super(Stem, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv4_1 =nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv4_2 = nn.Conv2d(in_channels=64, out_channels=96, kernel_size=3, stride=2)
        self.conv5_1 = nn.Sequential(nn.Conv2d(in_channels=160,out_channels=64, kernel_size=1),
                                            nn.Conv2d(in_channels=64, out_channels=96, kernel_size=3))
        self.conv5_2 = nn.Sequential(nn.Conv2d(in_channels=160, out_channels=64, kernel_size=1),
                                           nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(7,1),padding=(3,0)),
                                           nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(1,7),padding=(0,3)),
                                           nn.Conv2d(in_channels=64, out_channels=96, kernel_size=3),
                                           )
        self.conv6_1 = nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3, stride=2)
        self.conv6_2 = nn.MaxPool2d(kernel_size=3, stride=2)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = torch.cat([self.conv4_1(out), self.conv4_2(out)], 1)
        out = torch.cat([self.conv5_1(out), self.conv5_2(out)], 1)
        out = torch.cat([self.conv6_1(out), self.conv6_2(out)], 1)
        return out
        

In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_data = torch.randn(1,3,299,299)
net = Stem()
net.to(device)
input_data = input_data.to(device)
output = net(input_data)
print(output.size())

torch.Size([1, 384, 35, 35])


In [42]:
class InceptionV4A(nn.Module):
    def __init__(self):
        super(InceptionV4A, self).__init__()
        self.conv1_1 = nn.AvgPool2d(kernel_size=3, stride=1, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=384, out_channels=96, kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels=384, out_channels=96, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels=384, out_channels=64, kernel_size=1)
        self.conv3_2 = nn.Conv2d(in_channels=64, out_channels=96, kernel_size=3, padding=1)
        self.conv4_1 = nn.Conv2d(in_channels=384, out_channels=64, kernel_size=1)
        self.conv4_2 = nn.Conv2d(in_channels=64, out_channels=96, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(in_channels=96, out_channels=96, kernel_size=3, padding=1)
    def forward(self, x):
        out1 = self.conv1_2(self.conv1_1(x))
        out2 = self.conv2(x)
        out3 = self.conv3_2(self.conv3_1(x))
        out4 = self.conv4_3(self.conv4_2(self.conv4_1(x)))
        output = torch.cat([out1, out2, out3, out4], 1)
        return output
        

In [43]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_data = torch.randn(1,384,35,35)
net = InceptionV4A()
net.to(device)
input_data = input_data.to(device)
output = net(input_data)
print(output.size())

torch.Size([1, 384, 35, 35])


In [46]:
class InceptionV4B(nn.Module):
    def __init__(self):
        super(InceptionV4B, self).__init__()
        self.conv1_1 = nn.AvgPool2d(kernel_size=3, stride=1, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=1024, out_channels=128, kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels=1024, out_channels=384, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels=1024, out_channels=192, kernel_size=1)
        self.conv3_2 = nn.Conv2d(in_channels=192, out_channels=224, kernel_size=(1,7), padding=(0,3))
        self.conv3_3 = nn.Conv2d(in_channels=224, out_channels=256, kernel_size=(1,7), padding=(0,3))
        self.conv4_1 = nn.Conv2d(in_channels=1024, out_channels=192, kernel_size=1)
        self.conv4_2 = nn.Conv2d(in_channels=192, out_channels=192, kernel_size=(1,7), padding=(0,3))
        self.conv4_3 = nn.Conv2d(in_channels=192, out_channels=224, kernel_size=(7,1), padding=(3,0))
        self.conv4_4 = nn.Conv2d(in_channels=224, out_channels=224, kernel_size=(1,7), padding=(0,3))
        self.conv4_5 = nn.Conv2d(in_channels=224, out_channels=256, kernel_size=(7,1), padding=(3,0))
        self.conv4 = nn.Sequential(self.conv4_1, self.conv4_2, self.conv4_3, self.conv4_4, self.conv4_5)
    def forward(self, x):
        out1 = self.conv1_2(self.conv1_1(x))
        out2 = self.conv2(x)
        out3 = self.conv3_3(self.conv3_2(self.conv3_1(x)))
        out4 = self.conv4(x)
        output = torch.cat([out1, out2, out3, out4], 1)
        return output
        

In [47]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_data = torch.randn(1,1024,17,17)
net = InceptionV4B()
net.to(device)
input_data = input_data.to(device)
output = net(input_data)
print(output.size())

torch.Size([1, 1024, 17, 17])


In [48]:
class InceptionV4C(nn.Module):
    def __init__(self):
        super(InceptionV4C, self).__init__()
        self.conv1_1 = nn.AvgPool2d(kernel_size=3, stride=1, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=1536, out_channels=256, kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels=1536, out_channels=256, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels=1536, out_channels=384, kernel_size=1)
        self.conv3_2 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=(1,3), padding=(0,1))
        self.conv3_3 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=(3,1), padding=(1,0))
        self.conv4_1 = nn.Conv2d(in_channels=1536, out_channels=384, kernel_size=1)
        self.conv4_2 = nn.Conv2d(in_channels=384, out_channels=448, kernel_size=(1,3), padding=(0,1))
        self.conv4_3 = nn.Conv2d(in_channels=448, out_channels=512, kernel_size=(3,1), padding=(1,0))
        self.conv4_4 = nn.Conv2d(in_channels=512, out_channels=256, kernel_size=(3,1), padding=(1,0))
        self.conv4_5 = nn.Conv2d(in_channels=512, out_channels=256, kernel_size=(1,3), padding=(0,1))
    def forward(self, x):
        out1 = self.conv1_2(self.conv1_1(x))
        out2 = self.conv2(x)
        out3_1 = self.conv3_2(self.conv3_1(x))
        out3_2 = self.conv3_3(self.conv3_1(x))
        out4_1 = self.conv4_4(self.conv4_3(self.conv4_2(self.conv4_1(x))))
        out4_2 = self.conv4_5(self.conv4_3(self.conv4_2(self.conv4_1(x))))
        output = torch.cat([out1, out2, out3_1, out3_2, out4_1, out4_2], 1)
        return output
        

In [49]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_data = torch.randn(1,1536,8,8)
net = InceptionV4C()
net.to(device)
input_data = input_data.to(device)
output = net(input_data)
print(output.size())

torch.Size([1, 1536, 8, 8])


In [53]:
class ReductionA(nn.Module):
    def __init__(self):
        super(ReductionA, self).__init__()
        self.conv1 = nn.AvgPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=2)
        self.conv3_1 = nn.Conv2d(in_channels=384, out_channels=192, kernel_size=1)
        self.conv3_2 = nn.Conv2d(in_channels=192, out_channels=224, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(in_channels=224, out_channels=256, kernel_size=3, stride=2)
    def forward(self, x):
        out1 = self.conv1(x)
        out2 = self.conv2(x)
        out3 = self.conv3_3(self.conv3_2(self.conv3_1(x)))
        output = torch.cat([out1, out2, out3], 1)
        return output
        

In [54]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_data = torch.randn(1,384,35,35)
net = ReductionA()
net.to(device)
input_data = input_data.to(device)
output = net(input_data)
print(output.size())

torch.Size([1, 1024, 17, 17])


In [55]:
class ReductionB(nn.Module):
    def __init__(self):
        super(ReductionB, self).__init__()
        self.conv1 = nn.AvgPool2d(kernel_size=3, stride=2)
        self.conv2_1 = nn.Conv2d(in_channels=1024, out_channels=192, kernel_size=1)
        self.conv2_2 = nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3, stride=2)
        self.conv3_1 = nn.Conv2d(in_channels=1024, out_channels=256, kernel_size=1)
        self.conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(1,7), padding=(0,3))
        self.conv3_3 = nn.Conv2d(in_channels=256, out_channels=320, kernel_size=(7,1), padding=(3,0))
        self.conv3_4 = nn.Conv2d(in_channels=320, out_channels=320, kernel_size=3, stride=2)
        self.conv3 = nn.Sequential(self.conv3_1, self.conv3_2, self.conv3_3, self.conv3_4)
    def forward(self, x):
        out1 = self.conv1(x)
        out2 = self.conv2_2(self.conv2_1(x))
        out3 = self.conv3(x)
        output = torch.cat([out1, out2, out3], 1)
        return output
        

In [56]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_data = torch.randn(1,1024,17,17)
net = ReductionB()
net.to(device)
input_data = input_data.to(device)
output = net(input_data)
print(output.size())

torch.Size([1, 1536, 8, 8])


In [71]:
class GoogLeNetV4(nn.Module):
    def __init__(self):
        super(GoogLeNetV4, self).__init__()
        self.stem = Stem()
        
        self.inception_A1 = InceptionV4A()
        self.inception_A2 = InceptionV4A()
        self.inception_A3 = InceptionV4A()
        self.inception_A4 = InceptionV4A()
        self.inception_A = nn.Sequential(self.inception_A1, self.inception_A2, self.inception_A3, self.inception_A4)
        
        self.reduction_A = ReductionA()
        
        self.inception_B1 = InceptionV4B()
        self.inception_B2 = InceptionV4B()
        self.inception_B3 = InceptionV4B()
        self.inception_B4 = InceptionV4B()
        self.inception_B5 = InceptionV4B()
        self.inception_B6 = InceptionV4B()
        self.inception_B7 = InceptionV4B()
        self.inception_B = nn.Sequential(self.inception_B1, self.inception_B2, self.inception_B3, 
                                         self.inception_B4, self.inception_B5, self.inception_B6, self.inception_B7)
        
        self.reduction_B = ReductionB()
        
        self.inception_C1 = InceptionV4C()
        self.inception_C2 = InceptionV4C()
        self.inception_C3 = InceptionV4C()
        self.inception_C = nn.Sequential(self.inception_C1, self.inception_C2, self.inception_C3)

        self.pool = nn.AvgPool2d(kernel_size=7, stride=1, padding=3)
        
        self.dropout = nn.Dropout(0.8)

    def forward(self, x):
        network = self.stem(x)
        print('stem: ',network.size())
        network = self.inception_A(network)
        print('InceptionV4A: ',network.size())
        network = self.reduction_A(network)
        print('ReductionA: ',network.size())
        network = self.inception_B(network)
        print('InceptionV4B: ',network.size())
        network = self.reduction_B(network)
        print('ReductionB: ',network.size())
        network = self.inception_C(network)
        print('InceptionV4C: ',network.size())
        network = self.pool(network)
        print('AvgPool2d: ',network.size())
        network = self.dropout(network)
        print('dropout: ',network.size())
        return network

In [72]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_data = torch.randn(1,3,299,299)
net = GoogLeNetV4()
net.to(device)
input_data = input_data.to(device)
output = net(input_data)
print(output.size())

stem:  torch.Size([1, 384, 35, 35])
InceptionV4A:  torch.Size([1, 384, 35, 35])
ReductionA:  torch.Size([1, 1024, 17, 17])
InceptionV4B:  torch.Size([1, 1024, 17, 17])
ReductionB:  torch.Size([1, 1536, 8, 8])
InceptionV4C:  torch.Size([1, 1536, 8, 8])
AvgPool2d:  torch.Size([1, 1536, 8, 8])
dropout:  torch.Size([1, 1536, 8, 8])
torch.Size([1, 1536, 8, 8])
